In [13]:
import pandas as pd
import numpy as np
import pandasql as psql
import re
import os

In [34]:
os.listdir() 

['project_political_data - gov_party.csv',
 'main_analysis.ipynb',
 'house_leg_election_results.csv',
 'state_gov_control_2019.csv',
 'state_senate_election_results.json',
 '.DS_Store',
 'project_political_data - Center_for_repro_rights.csv',
 'merged_health_indicators.csv',
 'maternal_mortality_country_all_race_all_age_2019.csv',
 'abortion_laws_index.csv',
 'README.md',
 'ballotpedia_state_leg_scraper.ipynb',
 'senate_leg_election_results.csv',
 'project_proposal.Rmd',
 'project_proposal.pdf',
 '.ipynb_checkpoints',
 '.git',
 'maternal_mortality_county.ipynb',
 'project_political_data - States.csv']

In [6]:
#done 
maternal_mortality = pd.read_csv('maternal_mortality_country_all_race_all_age_2019.csv')
health_indicators = pd.read_csv('merged_health_indicators.csv')

abortion_laws_index = pd.read_csv('abortion_laws_index.csv')
state_gov_control = pd.read_csv('state_gov_control_2019.csv')


In [35]:
states = pd.read_csv('project_political_data - States.csv')

In [27]:
#fips, need to extract state from location 
maternal_mortality['state_name']= maternal_mortality['location_name'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0])
maternal_mortality['state_name'] = maternal_mortality['state_name'].str.replace(' ','_') 

In [33]:
maternal_mortality.head()

,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,sex_name,...,upper,lower,state_name,county_fips,low_income_pct,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct
0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,Female,...,0.000059,0.000031,Alabama,1001.0,22.7,3.5,71.8,32.1,1.1,3.0
1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,Female,...,0.000037,0.000022,Alabama,1003.0,21.0,19.4,78.3,28.0,1.2,3.3
2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,Female,...,0.000088,0.000048,Alabama,1005.0,43.5,0.0,67.4,41.2,1.1,4.6
3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,Female,...,0.000091,0.000050,Alabama,1007.0,35.1,0.0,66.4,37.5,1.1,3.1
4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,Female,...,0.000058,0.000032,Alabama,1009.0,28.0,3.4,72.3,32.8,1.2,2.9


In [30]:
#county_fips [low_income_pct,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct, hypertension_pre_pregnancy_pct]
#health_indicators.head()

health_indicators_target_var = health_indicators[['county_fips','low_income_pct','obgyn_provider_rate_100k','prenatal_care_first_trimester_pct','obesity_pre_pregnancy_pct','diabetes_pre_pregnancy_pct', 'hypertension_pre_pregnancy_pct' ]]

In [31]:
# merge health indicators 
maternal_mortality = pd.merge(maternal_mortality,health_indicators_target_var, how = 'left', left_on ='fips', right_on = 'county_fips')

In [36]:
maternal_mortality = pd.merge(maternal_mortality, states, how ='left', left_on = 'state_name', right_on ='State')

In [38]:
#'State Abbreviation`, 'guttmacher_index'
abortion_laws_index_target_var = abortion_laws_index[['State Abbreviation','guttmacher_index']]

maternal_mortality = pd.merge(maternal_mortality, abortion_laws_index_target_var, how ='left', left_on = 'State Abbreviation', right_on ='state')

In [41]:
state_gov_control_target_var = state_gov_control[['state','gop_trifecta']]

maternal_mortality = pd.merge(maternal_mortality, state_gov_control_target_var, how ='left', left_on = 'State', right_on ='state')

In [42]:
maternal_mortality.head()

,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,sex_name,...,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,State Abbreviation,State,guttmacher_index,state,gop_trifecta
0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,Female,...,3.5,71.8,32.1,1.1,3.0,AL,Alabama,most_restrictive,Alabama,1
1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,Female,...,19.4,78.3,28.0,1.2,3.3,AL,Alabama,most_restrictive,Alabama,1
2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,Female,...,0.0,67.4,41.2,1.1,4.6,AL,Alabama,most_restrictive,Alabama,1
3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,Female,...,0.0,66.4,37.5,1.1,3.1,AL,Alabama,most_restrictive,Alabama,1
4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,Female,...,3.4,72.3,32.8,1.2,2.9,AL,Alabama,most_restrictive,Alabama,1


In [44]:
maternal_mortality.isna().sum()

Unnamed: 0                            0
measure_id                            0
measure_name                          0
location_id                           0
location_name                         0
fips                                  0
race_id                               0
race_name                             0
sex_id                                0
sex_name                              0
age_group_id                          0
age_name                              0
cause_id                              0
cause_name                            0
year                                  0
metric_id                             0
metric_name                           0
val                                  31
upper                                31
lower                                31
state_name                            0
county_fips                           3
low_income_pct                       11
obgyn_provider_rate_100k              3
prenatal_care_first_trimester_pct     4


In [45]:
maternal_mortality.describe()

,Unnamed: 0,measure_id,location_id,fips,race_id,sex_id,age_group_id,cause_id,year,metric_id,...,upper,lower,county_fips,low_income_pct,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,gop_trifecta
count,3.010000e+03,3010.0,3010.000000,3010.000000,3010.0,3010.0,3010.0,3010.0,3010.0,3010.0,...,2979.000000,2979.000000,3007.000000,2999.000000,3007.000000,3006.000000,3006.000000,2923.000000,2976.000000,3010.000000
mean,3.849234e+06,1.0,3277.933555,30576.068771,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000049,0.000025,30571.898237,26.749350,9.755770,76.720393,31.970426,1.056244,2.299698,0.619934
std,1.161302e+05,0.0,10170.318218,15006.895800,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000017,0.000010,15005.211785,9.333029,13.431884,6.931233,4.876617,0.397371,1.041423,0.485484
min,3.648030e+06,1.0,574.000000,1001.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000010,0.000004,1001.000000,0.000000,0.000000,36.700000,8.700000,0.200000,0.100000,0.000000
25%,3.742232e+06,1.0,1366.250000,18169.500000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000037,0.000018,18170.000000,20.000000,0.000000,72.600000,29.400000,0.800000,1.500000,0.000000
50%,3.852783e+06,1.0,2186.500000,30004.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000046,0.000023,30003.000000,25.700000,3.900000,77.900000,32.300000,1.000000,2.100000,1.000000
75%,3.944098e+06,1.0,2942.750000,45074.500000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000057,0.000030,45072.000000,32.400000,16.300000,81.600000,34.900000,1.200000,2.900000,1.000000
max,4.044456e+06,1.0,94092.000000,56045.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000142,0.000088,56045.000000,73.300000,172.500000,93.000000,54.700000,10.700000,8.500000,1.000000


In [46]:
maternal_mortality.to_csv('maternal_mortality_complete_dataset.csv')